# Exemplo Árvore de Classificação

Neste exemplo iremos apresentar um modelo de árvore de classificação. O dataset utilizado será o dados_credito_small.csv.

O objetivo desta análise é desenvolver regras que buscarão classificar se um indivíduo irá pagar ou não pagar.

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(rpart)
library(visNetwork)

## Carregar dados

In [ ]:
dados_credito <- read_csv( file = "/home/vm-data-science/dados/dados_credito_small.csv" )
dados_credito %>% head

## Análises

Antes de iniciarmos a construção dos modelos é importante realizar um breve análise exploratória buscando comparar os indivíduos que pagaram e não pagaram.

### Análises em relação variáveis quantitativas

In [ ]:
dados_credito %>% 
    select( -id, -educacao, -tipo_emprego ) %>% 
    group_by( default ) %>% 
    summarise_if( is.numeric, mean )

### Análises em relação as variáveis categóricas

In [ ]:
dados_credito %>% 
    group_by( default ) %>% 
    count( educacao_cat ) %>% 
    spread( key = default, value = n )

In [ ]:
dados_credito %>% 
    group_by( default ) %>% 
    count( idade_cat ) %>% 
    spread( key = default, value = n )

In [ ]:
dados_credito %>% 
    group_by( default ) %>% 
    count( renda_cat ) %>% 
    spread( key = default, value = n )

### Treinamento do modelo

In [23]:
modelo_arvore_classificacao <- rpart( default ~ ., 
                                      data = dados_credito %>% select( -id ),
                                      parms = list( split = "gini") )

In [ ]:
summary(modelo_arvore_classificacao)

### Desenho da árvore

In [ ]:
modelo_arvore_classificacao %>% 
  visTree(., rules = TRUE,
          fallenLeaves = TRUE,
          width = "100%",
          height = "850" ) %>% 
  visOptions(highlightNearest = TRUE, 
             nodesIdSelection = FALSE) %>% 
  visLegend( width = 0.1 )

### Regras

In [26]:
source('/home/vm-data-science/R_scripts/tree_rules.R')

In [ ]:
tree_rules(modelo_arvore_classificacao, model_type = 'classification')

IF  [divida]>=12.45 AND [tipo_emprego]< 3.5 AND  ([renda_cat]=\'alta\' OR [renda_cat]=\'baixa\') THEN \'nao_pagou\' 

ELSEIF [divida]>=12.45 AND [tipo_emprego]< 3.5 AND  ([renda_cat]=\'baixa_media\' OR [renda_cat]=\'media_alta\') AND  ([idade_cat]=\'entre_30_40\') THEN \'nao_pagou\' 

ELSEIF [divida]>=12.45 AND [tipo_emprego]< 3.5 AND  ([renda_cat]=\'baixa_media\' OR [renda_cat]=\'media_alta\') AND  ([idade_cat]=\'abaixo_30\' OR [idade_cat]=\'acima_40\') THEN \'pagou\' 

ELSEIF [divida]>=12.45 AND [tipo_emprego]>=3.5 AND [divida]>=19.45 AND [renda]>=51.5 THEN \'nao_pagou\' 

ELSEIF [divida]>=12.45 AND [tipo_emprego]>=3.5 AND [divida]>=19.45 AND [renda]< 51.5 THEN \'pagou\' 

ELSEIF [divida]>=12.45 AND [tipo_emprego]>=3.5 AND [divida]< 19.45 AND [educacao]>=2.5 THEN \'nao_pagou\' 

ELSEIF [divida]>=12.45 AND [tipo_emprego]>=3.5 AND [divida]< 19.45 AND [educacao]< 2.5 THEN \'pagou\' 

ELSEIF [divida]< 12.45 AND [tipo_emprego]< 5.5 AND [idade]< 32.5 AND [outras_div]>=0.545 AND [outras_div]< 1.71 AND [outras_div]>=1.24 THEN \'nao_pagou\' 

ELSEIF [divida]< 12.45 AND [tipo_emprego]< 5.5 AND [idade]< 32.5 AND [outras_div]>=0.545 AND [outras_div]< 1.71 AND [outras_div]< 1.24 AND [outras_div]< 1.005 AND [idade]< 29.5 THEN \'nao_pagou\' 

ELSEIF [divida]< 12.45 AND [tipo_emprego]< 5.5 AND [idade]< 32.5 AND [outras_div]>=0.545 AND [outras_div]< 1.71 AND [outras_div]< 1.24 AND [outras_div]< 1.005 AND [idade]>=29.5 THEN \'pagou\' 

ELSEIF [divida]< 12.45 AND [tipo_emprego]< 5.5 AND [idade]< 32.5 AND [outras_div]>=0.545 AND [outras_div]< 1.71 AND [outras_div]< 1.24 AND [outras_div]>=1.005 THEN \'pagou\' 

ELSEIF [divida]< 12.45 AND [tipo_emprego]< 5.5 AND [idade]< 32.5 AND [outras_div]>=0.545 AND [outras_div]>=1.71 THEN \'pagou\' 

ELSEIF [divida]< 12.45 AND [tipo_emprego]< 5.5 AND [idade]< 32.5 AND [outras_div]< 0.545 THEN \'pagou\' 

ELSEIF [divida]< 12.45 AND [tipo_emprego]< 5.5 AND [idade]>=32.5 THEN \'pagou\' 

ELSEIF [divida]< 12.45 AND [tipo_emprego]>=5.5 THEN \'pagou\' 

END'